In [1]:
import numpy as np
import scipy as sc
import scipy.sparse
import scipy.io
import scipy.sparse.linalg
from scipy.sparse.linalg import spsolve_triangular
import matplotlib.pyplot as plt

In [125]:
# Параметры
gamma = -1
eps = 1e-5
lim_iter = 10000
# Подгружаем задачу
#open MTX
example = open("matrix_values.mtx")
A = scipy.io.mmread(example)
A.eliminate_zeros()
example.close()
A_csr = A.tocsr()
#

# open right part f
f_ex = np.loadtxt('vector_values.txt')
#
uu = np.loadtxt('u_dof_ids.txt', dtype = np.int64) 
pu = np.loadtxt('p_dof_ids.txt', dtype = np.int64) 
#
len_u = uu.size
len_p = pu.size

def CR(A, f, x, lim_iter, eps): 
    rr_count = np.array([])
    r = A.dot(x) - f
    rr = np.dot(r, r)
    Ar = A.dot(r)
    rAr = np.dot(Ar, r)
    p = np.copy(r)
    n_of_iter = 0
    rr_count = np.append(rr_count, np.sqrt(rr))
    while np.sqrt(rr)>eps and n_of_iter<lim_iter:
        Ap = A.dot(p)
        alpha = rAr/np.dot(Ap, Ap)
        x = x - alpha*p
        r = r - alpha*Ap
        rr = np.dot(r, r)
        Ar = A.dot(r)
        beta = rAr
        rAr = np.dot(Ar, r)
        beta = rAr/beta
        p = r + beta*p
        Ap = Ar + beta*Ap
        n_of_iter += 1
        rr_count = np.append(rr_count, np.sqrt(rr))
        
    return n_of_iter, rr_count, x


def A_dot(x, D, C, CT, gamma): #A_.dot(x)
    len_u = C.shape[0]
    len_p = C.shape[1]
    x_u = x[range(len_u)]
    x_p = x[range(len_u, len_u + len_p)]
    Dx_u = D.dot(x_u)
    CTx_u = CT.dot(x_u)
    C_CTx_u = C.dot(CTx_u)
    Cx_p = C.dot(x_p)
    y_u = Dx_u + gamma*C_CTx_u + Cx_p
    y_p = CT.dot(x_u)
    y = np.concatenate((y_u, y_p), axis = None)
    return y

def CR_CTPrecond(A, f, x, gamma, lim_iter, eps, u_ind, p_ind):
    p_ind = np.delete(p_ind, 43) # пока что так удалим условие Дирихле
    x_u = x[u_ind[:, None]]
    x_p = x[p_ind[:, None]]
    x = np.concatenate((x_u, x_p), axis = None)
    len_u = u_ind.size
    len_p = p_ind.size
    
    D = A[u_ind[:, None], u_ind[None, :]]
    CT = A[p_ind[:, None], u_ind[None, :]]
    C = A[u_ind[:, None], p_ind[None, :]]
    
    #уберем соответствующую строку и столбец из матрицы
    A = scipy.sparse.bmat([[D, C],[CT, None]])
    #
    f_u = f[u_ind[:, None]]
    f_p = f[p_ind[:, None]]
    f_u = f_u + gamma*C.dot(f_p)
    f = np.concatenate((f_u, f_p), axis = None)
    
    A_dott = lambda x: A_dot(x, D, C, CT, gamma) #шоб красивее было
    #CR
    rr_count = np.array([])
    r = A_dott(x) - f
    rr = np.dot(r, r)
    Ar = A_dott(r)
    rAr = np.dot(Ar, r)
    p = np.copy(r)
    n_of_iter = 0
    rr_count = np.append(rr_count, np.sqrt(rr))
    while np.sqrt(rr)>eps and n_of_iter<lim_iter:
        Ap = A_dott(p)
        alpha = rAr/np.dot(Ap, Ap)
        x = x - alpha*p
        r = r - alpha*Ap
        rr = np.dot(r, r)
        Ar = A_dott(r)
        beta = rAr
        rAr = np.dot(Ar, r)
        beta = rAr/beta
        p = r + beta*p
        Ap = Ar + beta*Ap
        n_of_iter += 1
        rr_count = np.append(rr_count, np.sqrt(rr))
    print(A_dott(x) - A.dot(x))
    return n_of_iter, rr_count, x

#приводим матрицу А и f для сравнения к такому же виду, как и в предобуславливателе
pu_ = np.delete(pu, 43) # пока что так удалим условие Дирихле
D = A_csr[uu[:, None], uu[None, :]]
CT = A_csr[pu_[:, None], uu[None, :]]
C = A_csr[uu[:, None], pu_[None, :]]
A_CR = scipy.sparse.bmat([[D, C],[CT, None]])
f_u = f_ex[uu[:, None]]
f_p = f_ex[pu_[:, None]]
f_CR = np.concatenate((f_u, f_p), axis = None)
x_CR = np.array([0 for i in range(len_u + len_p - 1)])
x = np.array([0 for i in range(len_u + len_p)])



n_of_iter, rr_count, x_CT = CR_CTPrecond(A_csr, f_ex, x, gamma, lim_iter, eps, uu, pu)
n_of_iter2, rr_count2, x2 = CR(A_CR, f_CR, x_CR, lim_iter, eps)

[0. 0. 0. ... 0. 0. 0.]


In [126]:
n_of_iter, n_of_iter2, np.sum(abs(x_CT-x2))

(1461, 1315, 0.3954143109071933)

In [110]:
x = np.array([1 for i in range(len_u + len_p)])
A_dott = lambda x: A_dot(x, D, C, CT, gamma)
print(x_CT - x2)

[0.         0.         0.         ... 1.6403572  1.63125682 0.29972971]


In [122]:
for i in range(x2.size):
    print(abs(x_CT[i]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09780951380519534
0.09780835134049339
1.1415324124889668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
0.1597109583679374
0.1597126274896859
1.1568138302178141
0.0
0.0
0.0
0.0
0.0
0.0
0.06938114130208388
0.04633105401305488
0.8265135299089481
0.046332953473738316
0.06938111004511949
0.8265118524945015
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
0.0
0.0
0.0
0.12143044629848855
0.1478290062149893
0.9282272865540395
0.14782618897684388
0.12143058583994465
0.9282220378530911
0.03409943796824835
0.03409703516980748
0.6439745356713733
0.1173798558877217
0.1173834875557789
0.7935432962687955
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
6.179454919709018e-08
0.11140993702793742
1.2355891706397277
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
0.0
0.0
0.0
0.0
0.0
1.0000000000019862
9.202819326659641e-07
0.18016498229584052
1.1390708361

In [119]:
np.linalg.norm(A_CR.dot(x_CT) - f_CR), np.linalg.norm(A_CR.dot(x2) - f_CR)

(9.292751116452291e-06, 9.817721875485036e-06)

In [189]:
#Бидиагонализация Голуба-Кахана с gamma = 0
# Параметры
niu = 0.0001
gamma = 0
eps = 1e-5
lim_iter = 10000
# Подгружаем задачу
#open MTX
example = open("matrix_values.mtx")
A = scipy.io.mmread(example)
A.eliminate_zeros()
example.close()
A_csr = A.tocsr()
#

# open right part f
f_ex = np.loadtxt('vector_values.txt')
#
uu = np.loadtxt('u_dof_ids.txt', dtype = np.int64) 
pu = np.loadtxt('p_dof_ids.txt', dtype = np.int64) 
#
len_u = uu.size
len_p = pu.size

pu_ = np.delete(pu, 43) # пока что так удалим условие Дирихле
M = A_csr[uu[:, None], uu[None, :]]
CT = A_csr[pu_[:, None], uu[None, :]]
C = A_csr[uu[:, None], pu_[None, :]]
A_CR = scipy.sparse.bmat([[D, C],[CT, None]])
g = f_ex[uu]
r = f_ex[pu_]
Mrg = scipy.sparse.linalg.bicgstab(M, g, tol=1e-10)[0]
b = r - (CT@Mrg)
f_CR = np.concatenate((g, r), axis = None)
N = (1/niu)*scipy.sparse.identity(len_p - 1)
Nr = (niu**2)*N

def converge(M, C, u, p, b, eps):
    A = scipy.sparse.bmat([[M, C],[C.transpose(), None]])
    f1 = np.array([0 for i in range(u.shape[0])])
    f = np.concatenate((f1, b), axis = None)
    x = np.concatenate((u, p), axis = None)
    r = np.linalg.norm(A@x - f)
    return 1 if r < eps else 0, r

def GCB(M, A, N, Nr, b, lim_iter, eps):
    conv = False
    AT = A.transpose()
    eps_bicg = 1e-8
    en_norm = lambda x, M: x@(M@x)
    Mr_dot = lambda x: scipy.sparse.linalg.bicgstab(M, x, tol=eps_bicg)[0]

    k = 0
    beta = en_norm(b, Nr)
    q = (1/beta)*(Nr@b)
    w = Mr_dot(A@q)
    print(np.linalg.norm(M@w-A@q))
    alpha = en_norm(w, M)
    v = (1/alpha)*w
    zetta = beta/alpha
    d = (1/alpha)*q
    p = -zetta*d
    u = zetta*v
    while conv == False and k < lim_iter:
        k+=1
        g = Nr@(AT@v - alpha*(N@q))
        beta = en_norm(g, N)
        q = (1/beta)*g
        w = Mr_dot(A@q - beta*M@v)
 #      print(np.linalg.norm(M@w-A@q + beta*M@v))
        alpha = en_norm(w, M)
        v = (1/alpha)*w
        zetta = -zetta*beta/alpha
        d = (1/alpha)*(q - beta*d)
        u = u + zetta*v
        print(zetta)
        p = p - zetta*d
        conv = converge(M, A, u, p, b, eps)[0]      
    return u, p


x, p = GCB(M, C, N, Nr, b, lim_iter, eps)


9.62311656904594e-10
-0.7145083203659999
0.004856411626186263
-3.3008340301777303e-05
2.2435300248984122e-07
-1.5248955044245188e-09
1.0364498241611578e-11
-7.044602302701735e-14
4.788116168745952e-16
-3.2544145807180636e-18
2.2119793864045518e-20
-1.5034509849154501e-22
1.0218742897502813e-24
-6.945534471730456e-27
4.720781175336137e-29
-3.208647944931125e-31
2.180872456032415e-33
-1.4823080474741889e-35
1.0075037363600035e-37
-6.8478598674559165e-40
4.654393138197279e-42
-3.1635249417858523e-44
2.150202993566865e-46
-1.4614624507160088e-48
9.93335281181957e-51
-6.751558895843101e-53
4.588938739544807e-55
-3.119036519375073e-57
2.1199648465564227e-59
-1.4409100129078815e-61
9.793660819521789e-64
-6.656612238431508e-66
4.5244048488981765e-68
-3.0751737530994764e-70
2.0901519490789234e-72
-1.4206466111294972e-74
9.655933363808588e-77
-6.56300084845914e-79
4.460778520446395e-81
-3.031927843876984e-83
2.0607583205244686e-85
-1.4006681804460797e-87
9.520142815901247e-90
-6.470705947041331e

KeyboardInterrupt: 